In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=2, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr

In [5]:
import motor_jacobian

# Compute Jacobian

In [6]:
mot = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(mot.log())

Dll: [ 0 -0.5236 0 -0.7152 -0.5 -0.1917 ]


In [7]:
p = vsr.Vec(0,0,0).null()
p.comm(vsr.Lin(vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,0)).dual())

Dlp: [ 0 0 1 0 ]

In [8]:
mot = vsr.Mot(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0)

In [9]:
dll = vsr.Dll(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mot0 = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [10]:
mot.comm(dll)

Mot: [ 0 0 -3 2 -5 4 0 0 ]

In [11]:
mot.acomm(dll)

Mot: [ -1 0 0 0 0 0 -7 6 ]

In [12]:
mot.acomm(mot0)

Mot: [ 0 1 2 3 4 5 6 7 ]

In [13]:
mot.acomm(dll) - mot.comm(dll) 

Mot: [ -1 0 3 -2 5 -4 -7 6 ]

In [14]:
j = np.zeros((8,6))
for i in range(6):
    dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    dll[i] = 1.0
    j[:,i] = np.array(mot.acomm(dll)) - np.array(mot.comm(dll)) 
print(j)

[[-1. -2. -3.  0.  0.  0.]
 [ 0. -3.  2.  0.  0.  0.]
 [ 3.  0. -1.  0.  0.  0.]
 [-2.  1.  0.  0.  0.  0.]
 [ 5.  6. -7.  0. -1. -2.]
 [-4.  7.  6.  1.  0. -3.]
 [-7. -4. -5.  2.  3.  0.]
 [ 6. -5.  4.  3. -2.  1.]]


In [15]:
j = np.zeros((8,6))
for i in range(6):
    dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    dll[i] = 1.0
#     print(dll)
    j[:,i] = np.array(dll.comm(mot)) + np.array(dll.acomm(mot)) 
print(j)

[[-1. -2. -3.  0.  0.  0.]
 [ 0. -3.  2.  0.  0.  0.]
 [ 3.  0. -1.  0.  0.  0.]
 [-2.  1.  0.  0.  0.  0.]
 [ 5.  6. -7.  0. -1. -2.]
 [-4.  7.  6.  1.  0. -3.]
 [-7. -4. -5.  2.  3.  0.]
 [ 6. -5.  4.  3. -2.  1.]]


In [16]:
dll = vsr.Dll(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)
dll * mot

Mot: [ -1 0 3 -2 5 -4 -7 6 ]

In [17]:
dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
jac_exp = motor_jacobian.jacobian_exp(mot, dll)
print(jac_exp)
print(jac_exp.shape)

[[-1. -2. -3.  0.  0.  0.]
 [ 0. -3.  2.  0.  0.  0.]
 [ 3.  0. -1.  0.  0.  0.]
 [-2.  1.  0.  0.  0.  0.]
 [ 5.  6. -7.  0. -1. -2.]
 [-4.  7.  6.  1.  0. -3.]
 [-7. -4. -5.  2.  3.  0.]
 [ 6. -5.  4.  3. -2.  1.]]
(8, 6)


In [18]:
np.allclose(j, jac_exp)

True

In [19]:
jac_polar = motor_jacobian.jacobian_polar(mot, mot0)
print(jac_polar)
print(jac_polar.shape)

[[ 0.24 -0.02 -0.03 -0.05  0.    0.    0.    0.  ]
 [-0.02  0.24 -0.03 -0.05  0.    0.    0.    0.  ]
 [-0.03 -0.03  0.19 -0.1   0.    0.    0.    0.  ]
 [-0.05 -0.05 -0.1   0.1   0.    0.    0.    0.  ]
 [ 0.3  -0.37  0.14 -0.4   0.1   0.1  -0.05  0.05]
 [-0.33  0.11 -0.34 -0.14  0.1   0.19  0.03 -0.03]
 [ 0.02 -0.22 -0.11 -0.37 -0.05  0.03  0.24  0.02]
 [-0.23 -0.02 -0.33 -0.3   0.05 -0.03  0.02  0.24]]
(8, 8)


In [20]:
dll.exp()

Mot: [ 1 0 0 0 0 0 0 0 ]

In [21]:
mot = vsr.Lin(vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,0)).dual().exp()
print(mot)
p = vsr.Vec(0,0,0).null()

Mot: [ 0.54 0 0 -0.84 0 0 0.84 0 ]


In [22]:
print(motor_jacobian.diff_point(mot,p))

[[ 0.    0.    0.    0.   -1.08  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.08]
 [ 0.    0.   -1.68  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]]


In [23]:
p = vsr.Vec(0,0,0).null()
p.comm(vsr.Lin(vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,0)).dual())

Dlp: [ 0 0 1 0 ]

In [24]:
p.comm(mot)

Pnt: [ 0 0 0.8415 0 0 ]

In [25]:
p.acomm(mot)

Pnt: [ 0 0 0 0.5403 0 ]

In [476]:
a = vsr.Vec(1,0,0).null()
b = a.spin(vsr.Rot(vsr.Biv(1,0,0) * pi/4))
print(a)
print(b)
m = vsr.Vec(1,2,0).trs() * vsr.Rot(vsr.Biv(0,0,0) * pi/6)
print(m)
c = a.spin(m)
print(c)
points_a = [a]
points_b = [b]

Pnt: [ 1 0 0 1 0.5 ]
Pnt: [ 2.22e-16 1 0 1 0.5 ]
Mot: [ 1 0 0 0 -0.5 -1 0 0 ]
Pnt: [ 2 2 0 1 4 ]


In [477]:
c.vec() - b.vec()

Vec: [ 2 1 0 ]

In [478]:
# m = vsr.Mot(1,0,0,0,0,0,0,0)
dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
jac_exp = motor_jacobian.jacobian_exp(m, dll)
print(jac_exp)
jac_s = motor_jacobian.diff_cost(m, a, b)
print(jac_s)
np.dot(jac_s, jac_exp)

[[ 0.   0.   0.   0.   0.   0. ]
 [ 1.   0.   0.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.   0. ]
 [ 0.   0.   1.   0.   0.   0. ]
 [-1.   0.   0.   1.   0.   0. ]
 [ 0.5  0.   0.   0.   1.   0. ]
 [ 0.   0.5  1.   0.   0.   1. ]
 [ 0.   1.  -0.5  0.   0.   0. ]]
[[ 3.  2.  0.  0. -2.  0.  0.  0.]
 [ 2. -3.  0.  0.  0. -2.  0.  0.]
 [ 0.  0. -3. -2.  0.  0. -2.  0.]]


array([[ 4.,  0.,  0., -2.,  0.,  0.],
       [-4.,  0.,  0.,  0., -2.,  0.],
       [ 0., -4., -4.,  0.,  0., -2.]])

In [472]:
c = a.spin(m)

In [436]:
np.array(c.vec() - b.vec())

array([ 0.5 , -0.13,  0.  ])

In [437]:
import scipy.optimize as opt

In [438]:
motor = vsr.Vec(1,0,0).trs() * vsr.Rot(vsr.Biv(0,0,0).unit() * np.pi/6.0)
print(motor)

Mot: [ 1 0 0 0 -0.5 0 0 0 ]


In [423]:
n_points = 3
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.9, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [424]:
def func(arr):
    return np.array([np.array(a.spin(vsr.Dll(*arr).exp()).vec() - b.vec()) 
                     for a, b in zip(points_a, points_b)]).ravel()

In [425]:
init_dll = np.array([0,0,0,0,0,0]).reshape(6,1)
current_mot = np.array([1,0,0,0,0,0,0,0]).reshape(8,1)

In [426]:
its = 0

In [444]:
def jac(arg):
    global its
    global current_mot
    current_mot = np.array(vsr.Dll(*arg).exp() * vsr.Mot(*current_mot))
    dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    
    jac_exp = motor_jacobian.jacobian_exp(vsr.Mot(*current_mot), dll)
    print(jac_exp)

    js_ = np.array([motor_jacobian.diff_cost(vsr.Mot(*current_mot), a, b)
                   for a, b in zip(points_a, points_b)])
    
    print(js_)
    
    js = np.array([np.dot(motor_jacobian.diff_cost(vsr.Mot(*current_mot), a, b), jac_exp)
                   for a, b in zip(points_a, points_b)])
    
    print(js)
    
    js = js.reshape(-1,6)
    return js

In [445]:
print(func(init_dll))
jac(init_dll)

[ 0.5  -0.87  0.  ]
[[ 0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]]
[[[ 2.  0.  0.  0. -2.  0.  0.  0.]
  [ 0. -2.  0.  0.  0. -2.  0.  0.]
  [ 0.  0. -2.  0.  0.  0. -2.  0.]]]
[[[ 0.  0.  0. -2.  0.  0.]
  [-2.  0.  0.  0. -2.  0.]
  [ 0. -2.  0.  0.  0. -2.]]]


array([[ 0.,  0.,  0., -2.,  0.,  0.],
       [-2.,  0.,  0.,  0., -2.,  0.],
       [ 0., -2.,  0.,  0.,  0., -2.]])

In [431]:
out = opt.leastsq(func, init_dll, Dfun=jac, full_output=False)
print(out)

TypeError: Improper input: N=6 must not exceed M=3

In [419]:
estimated_motor = vsr.Dll(*out[0]).exp()
print(estimated_motor)

Mot: [ 1 0 0 0 -0.5 -6.4e-18 1.5e-17 0 ]


In [409]:
print(motor)

Mot: [ 1 0 0 0 -0.5 -1 -1.5 0 ]


In [316]:
jac(init_dll)

array([[ 3.4 ,  0.37,  0.  , -2.  ,  0.  ,  0.  ],
       [-2.94, -0.  ,  0.37,  0.  , -2.  , -0.  ],
       [ 0.  , -2.94, -3.4 ,  0.  ,  0.  , -2.  ],
       ..., 
       [ 1.9 ,  0.96,  0.  , -2.  ,  0.  ,  0.  ],
       [-1.65, -0.  ,  0.96,  0.  , -2.  , -0.  ],
       [ 0.  , -1.65, -1.9 ,  0.  ,  0.  , -2.  ]])